In [25]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/22 22:02:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-22 22:13:44--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.106.67
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.106.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  26.7MB/s    in 18s     

2022-02-22 22:14:02 (40.0 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [13]:
!wc -l fhvhv_tripdata_2021-01.csv

 11908469 fhvhv_tripdata_2021-01.csv


In [15]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [18]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [23]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [22]:
!wc -l head.csv

     101 head.csv


In [26]:
df_pandas = pd.read_csv('head.csv')

In [27]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [33]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [34]:
from pyspark.sql import types

In [38]:
schema = types.StructType(
    [
        types.StructField('hvfhs_license_num', types.StringType(), True),
        types.StructField('dispatching_base_num', types.StringType(), True),
        types.StructField('pickup_datetime', types.TimestampType(), True),
        types.StructField('dropoff_datetime', types.TimestampType(), True),
        types.StructField('PULocationID', types.IntegerType(), True),
        types.StructField('DOLocationID', types.IntegerType(), True),
        types.StructField('SR_Flag', types.StringType(), True)
    ]
)

In [39]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [43]:
df = df.repartition(24)

In [44]:
df.write.parquet('fhvhv/2021/01')

22/02/22 22:55:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/22 22:55:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/22 22:55:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/22 22:55:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/02/22 22:55:38 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [45]:
df = spark.read.parquet('fhvhv/2021/01/')

In [48]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [51]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 17:09:16|2021-01-03 17:20:33|         244|          74|
|2021-01-04 07:58:15|2021-01-04 08:13:32|         238|         237|
|2021-01-02 13:44:12|2021-01-02 14:13:35|         121|         244|
|2021-01-01 21:49:21|2021-01-01 21:56:11|          92|          92|
|2021-01-01 08:44:43|2021-01-01 08:56:24|         177|          77|
|2021-01-04 08:50:48|2021-01-04 08:59:27|          28|         215|
|2021-01-01 02:57:50|2021-01-01 03:10:05|          91|          39|
|2021-01-03 08:37:23|2021-01-03 08:44:19|         181|         106|
|2021-01-04 23:17:37|2021-01-04 23:37:24|         198|         211|
|2021-01-01 07:25:23|2021-01-01 07:37:43|          28|         132|
|2021-01-01 14:09:55|2021-01-01 14:25:54|         185|         241|
|2021-01-04 18:14:57|2021-01-04 18:41:20|       

In [52]:
from pyspark.sql import functions as F

In [53]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_date|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+------------+
|           HV0003|              B02869|2021-01-03 17:09:16|2021-01-03 17:20:33|         244|          74|   null| 2021-01-03|  2021-01-03|
|           HV0003|              B02879|2021-01-04 07:58:15|2021-01-04 08:13:32|         238|         237|   null| 2021-01-04|  2021-01-04|
|           HV0003|              B02879|2021-01-02 13:44:12|2021-01-02 14:13:35|         121|         244|   null| 2021-01-02|  2021-01-02|
|           HV0003|              B02872|2021-01-01 21:49:21|2021-01-01 21:56:11|          92|          92|   null| 2021-01-01|  2021-01-01|
|           HV0003| 

In [54]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02869|2021-01-03 17:09:16|2021-01-03 17:20:33|         244|          74|   null|
|           HV0003|              B02879|2021-01-04 07:58:15|2021-01-04 08:13:32|         238|         237|   null|
|           HV0003|              B02879|2021-01-02 13:44:12|2021-01-02 14:13:35|         121|         244|   null|
|           HV0003|              B02872|2021-01-01 21:49:21|2021-01-01 21:56:11|          92|          92|   null|
|           HV0003|              B02682|2021-01-01 08:44:43|2021-01-01 08:56:24|         177|          77|   null|
|           HV0003|              B02883|2021-01-04 08:50:48|2021-01-04 08:59:27|

In [55]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [56]:
crazy_stuff('B02884')

's/b44'

In [57]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [58]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b35| 2021-01-03|  2021-01-03|         244|          74|
|  e/b3f| 2021-01-04|  2021-01-04|         238|         237|
|  e/b3f| 2021-01-02|  2021-01-02|         121|         244|
|  e/b38| 2021-01-01|  2021-01-01|          92|          92|
|  a/a7a| 2021-01-01|  2021-01-01|         177|          77|
|  a/b43| 2021-01-04|  2021-01-04|          28|         215|
|  e/9ce| 2021-01-02|  2021-01-02|           3|          78|
|  s/b44| 2021-01-01|  2021-01-01|          91|          39|
|  e/9ce| 2021-01-01|  2021-01-01|          87|         211|
|  e/b35| 2021-01-03|  2021-01-03|         181|         106|
|  e/acc| 2021-01-04|  2021-01-04|         198|         211|
|  e/b48| 2021-01-01|  2021-01-01|          28|         132|
|  e/b42| 2021-01-01|  2021-01-01|         185|         241|
|  e/acc| 2021-01-04|  2